# Time-Weighted Vector Store Retriever

Time-Weighted Vector Store Retriever 是一种特殊的检索器，它是一种结合语义相似性和时间衰减的检索方法。
不是基于文档创建时间，而是基于最后访问时间；频繁访问的对象会保持"新鲜度"。

检索分数计算公式为：
```
semantic_similarity + (1.0 - decay_rate) ^ hours_passed

```

In [33]:
from datetime import datetime, timedelta

from langchain_core.documents import Document
from langchain_ollama import OllamaEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_community.vectorstores import FAISS
from langchain_community.docstore import InMemoryDocstore
from langchain.retrievers import TimeWeightedVectorStoreRetriever 
import faiss

## 1. 声明TimeWeightedVectorStoreRetriever
--------------
由于Chroma向量数据库meta元数据不支持datetime类型，这也意味着如果要使用lanchain的时间衰减检索法，可能要就放弃chroma数据库，这个例子里使用了FAISS数据库。

每个数据库在功能和性能上都有细微的差别，我们需要按自身的需求选择合适业务场景的组件服务。

In [41]:
# Define your embedding model
embeddings_model = OpenAIEmbeddings()
# Initialize the vectorstore as empty
embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model, index, InMemoryDocstore({}), {})
retriever = TimeWeightedVectorStoreRetriever(
    vectorstore=vectorstore, decay_rate=0.0000000000000000000000001, k=1
)

## 2. 添加文档

In [43]:
# 这里默认last_accessed是当前时间
retriever.add_documents([Document(page_content="I love fruit.")])

# 指定last_accessed时间为昨天
yesterday = datetime.now() - timedelta(days=1)
retriever.add_documents([
    Document(page_content="I love programming.", metadata={"last_accessed": yesterday}),
    Document(page_content="I love basketball.", metadata={"last_accessed": yesterday}),
    Document(page_content="I love baseball.", metadata={"last_accessed": yesterday})
])

['cd968daf-9db0-4ffe-8ccf-7b8c1fe62920',
 '8f02225f-c358-4e7f-8762-0a5cf69537a9',
 '3753c034-3d23-40b4-a946-c472e7715a00']

## 3. 语义检索

In [44]:
retriever.invoke("What do I like?")

[Document(metadata={'last_accessed_at': datetime.datetime(2025, 2, 6, 14, 24, 26, 211445), 'created_at': datetime.datetime(2025, 2, 6, 14, 14, 36, 877739), 'buffer_idx': 0}, page_content='I love fruit.')]